# Ab Initio Molecular Dynamics using VASP and ASE

Authors: <br>
[Prof. Elvis do A. Soares](https://github.com/elvissoares) <br>
[PhD. Hugo de L. C. Neto](https://github.com/hugo-neto) <br>

Contacts: <br>
[elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/) <br>
[hneto@peq.coppe.ufrj.br](mailto:hneto@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [ ]:
import os
# Definindo o path para os arquivos de potencial de pseudopotenciais do VASP
# Certifique-se de que o caminho esteja correto para o seu sistema
os.environ['VASP_PP_PATH'] = '/home/hugo/Documents/Programs/VASP/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std_gpu_mlff'
os.environ['NO_STOP_MESSAGE'] = '1' # to avoid warning from mpirun

In [ ]:
# Importing the VASP calculator of ASE
from ase.calculators.vasp import Vasp

from ase import Atoms
from ase.build import molecule
from ase.io import write, read
from ase.visualize import view

import numpy as np

import os

# AIMD for Si

In [ ]:
# 1) Get the structure
#    - either read your existing POSCAR
atoms = read('POSCAR')      # or build it with ase.build.bulk, etc.

In [ ]:
# 2) Attach a VASP calculator with the INCAR-equivalent settings
calc = Vasp(
    directory='vasp_run',  # directory where all input files will go
    xc='PBE',
    encut=300,
    kpts=(2, 2, 2),        # Monkhorst–Pack grid -> KPOINTS
    ismear=-1,
    sigma=0.0258,
    ediff=1e-6,
    isym=0,
    ivdw=10,
    #Print conf
    lwave=False, lcharg=False,lreal=False,
    #MD
    ibrion=2,   #MD (treat ionic degrees of freedom)
    nsw=10000,  #no of ionic steps
    potim=2,    #MD time step in fs
    #Thermostat
    mdalgo=3,
    langevin_gamma=1,
    langevin_gamma_l=10,
    pmass=10,
    tebeg=400,
    isif=3,
    #ML
    ml_lmlff=True,
    ml_mode="train"
)

# 3a) If you ONLY want to generate the input files:
calc.write_input(atoms)
# This writes: INCAR, POSCAR, KPOINTS, POTCAR into vasp_run/

# 3b) If you also want to actually run VASP:
#atoms.calc = calc
#energy = atoms.get_potential_energy()   # writes input files and runs VASP

/home/hugo/Downloads/COQ-878/FinalWork
Atoms(symbols='C24H40O20', pbc=True, cell=[[10.374574938549959, -0.627959351297126, -0.4187754904260583], [-0.5031479190238899, 8.45796954566284, 0.365981129078457], [-0.2349863896669763, -0.5952008631941367, 7.6642064917338875]], momenta=...)


In [ ]:
AIMD_Out_Dir = "AIMD_Cellulose_300K_350K_Test"

In [ ]:
calc_aimd = Vasp(
    directory=AIMD_Out_Dir,
    xc='PBE',
    encut=450, 
    ismear=0, sigma=0.05,           # Gaussian smearing 
    prec = 'Normal',                # Normal precision
    algo = 'Fast',                  # Fast electronic minimization
    ibrion=0,                       # Molecular Dynamics
    isym=0,                         # Symmetry off (strongly recommended for MD)
    potim=0.5,                      # timestep 0.5 fs
    nsw=10, isif=2,                 # Number of MD steps
    mdalgo=4, setups='recommended', # Nosé-Hoover-Chain thermostat
    istart=0, lreal = 'Auto', lwave=False, lcharg=False,
    tebeg=300, teend=350,          # Start (300 K) and Final (350K) temperature 
    ivdw=12,                       # D3(BJ) van der Waals correction
    atoms=atoms
)

calc_aimd.calculate(atoms)

| N  | t (fs)| Elapsed Time (min)|
| ---|-------|-------------------|
| 84 | XXXX  | XXX               |
| XX | XXXX  | XX                |

In [ ]:
# Load all frames from XDATCAR
traj = read(f"{AIMD_Out_Dir}/XDATCAR", index=':')

print(f"Trajectory has {len(traj)} frames")
#print(traj[0])  # first configuration

write(f"{AIMD_Out_Dir}/aimd.traj", traj)        # ASE binary trajectory
write(f"{AIMD_Out_Dir}/aimd.xyz", traj)         # Plain XYZ for external tools
write(f"{AIMD_Out_Dir}/aimd.pdb", traj)         # PDB format if preferred

Trajectory has 10 frames
Atoms(symbols='C24H40O20', pbc=True, cell=[[10.374575, -0.627959, -0.418775], [-0.503148, 8.45797, 0.365981], [-0.234986, -0.595201, 7.664206]])
